## Time-domain CW Received Power Calculation

In [1]:
def abs2(x):
    return np.real(np.multiply(x,np.conj(x)))


def printRSSMeasurements(avgRxPowers):
    for txname, measurements in avgRxPowers.items():
        print("TX =", txname)
        for rxname, rss in measurements.items():
            if rss < 0:
                print("    RX =", rxname, ', RSS = %.2f' % rss)

def printRSSMeasurementsGrid(avgRxPowers, names):
    maxchars = max([len(x) for x in names])
    # Print header row with names of transmitters
    print(' '*maxchars, ' '.join(format(x, ">22s") for x in names))
    for rowname in names:
        print( "%22s" % rowname, end='')
        for colname in names:
            print( "%23.2f" % avgRxPowers[rowname][colname], end='')
        print()

def calcRxPowerAllLinks(rx_data, txrxloc, rxlocs, txlocs, samp_rate=250000, repeats=4):

    # Design parameters: You can change these parameters depending on the
    # (normalized) bandwidth of the signal you have.
    stopband_attenuation = 30.0
    transition_bandwidth = 0.002 # 100 Hz / 500000 = 0.0002
    cutoff_norm = 0.01  # normalized bandwidth of the passband
    # What order filter do we need?
    filterN, beta = signal.kaiserord(stopband_attenuation, transition_bandwidth)
    # Create the filter coefficients
    taps = signal.firwin(filterN, cutoff_norm, window=('kaiser', beta))
    print('Filter tap length:', len(taps))

    # Initialize a 2D dictionary array, each containing an array of the "repeats" number of received powers.
    allRxPowers = {y: dict( {w: np.zeros(repeats) for w in rxlocs} ) for y in txlocs}
    avgRxPowers = {y: dict( {w: 0 for w in rxlocs} ) for y in txlocs}

    # Loop through each measurement (RX, TX, and repetition) in the rx_data structure.
    for txname in rx_data:
        for i in range(0, len(rx_data[txname])):
            rxname = txrxloc[txname][i]
            for j in range(0, len(rx_data[txname][i])):
                #print('TX: {} RX: {} rep: {}'.format(txname, txrxloc[txname][i], j))

                # Filter the out-of-band noise
                filtered_signal = signal.lfilter(taps, 1.0, rx_data[txname][i][j])
                # Compute the average linear power in the filtered signal
                power_linear = np.mean(abs2(filtered_signal))
                # Convert to dB and store it
                allRxPowers[txname][rxname][j] = 10.0*np.log10(power_linear)

            # Store the average power over all repetitions
            avgRxPowers[txname][rxname] = np.mean(allRxPowers[txname][rxname])
            
    return avgRxPowers, allRxPowers

avgRxPowers, allRxPowers = calcRxPowerAllLinks(rx_data, txrxloc, rxlocs, txlocs, samp_rate, rxrepeat)
# Print out the results in a tabular format
printRSSMeasurementsGrid(avgRxPowers, rxlocs)

NameError: name 'rx_data' is not defined